In [ ]:
%load_ext autoreload
%autoreload 2


import pandas as pd
import json
import os


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
os.getcwd()

'd:\\ml_projects\\aleph_alpha\\aa-case-study-ai-solutions-engineer-main'

In [3]:
old_tickets=[]
for file in os.listdir('data\old_tickets'):
    if file.endswith('.csv'):
        old_tickets.append(pd.read_csv(os.path.join('data\old_tickets',file)))
    if file.endswith('.xlsx'):
        old_tickets.append(pd.read_excel(os.path.join('data\old_tickets',file)))
    if file.endswith('.json'):
        old_tickets.append(pd.read_json(os.path.join('data\old_tickets',file)))
    

In [4]:
old_tickets_df=pd.concat(old_tickets,axis=0)

In [5]:
old_tickets_df.columns

Index(['Ticket ID', 'Issue', 'Category', 'Resolution', 'Date', 'Agent Name',
       'Resolved', 'Description'],
      dtype='object')

In [6]:
old_tickets_df

,Ticket ID,Issue,Category,Resolution,Date,Agent Name,Resolved,Description
0,TCKT-1010,Email synchronization error,Software,Reconfigured email settings,2024-01-20,Alice Johnson,True,"Emails not syncing correctly across devices, c..."
1,TCKT-1011,VPN disconnection issues,Network,VPN settings updated,2024-01-21,Bob Smith,False,Repeated VPN disconnections impacting remote a...
2,TCKT-1012,Password reset issue,Account Management,Password reset successfully,2024-01-22,Cathy Brown,True,User unable to reset password due to system er...
3,TCKT-1013,Request for software installation rights,Software,Installation permissions granted,2024-01-23,David Clark,False,Employee requesting installation rights for ne...
4,TCKT-1044,Printer connectivity problem,Hardware,Printer driver reinstalled,2024-01-24,Eva Adams,True,Printer not connecting to any devices in the o...
5,TCKT-1045,Laptop overheating and shutting down,Hardware,Cooling system cleaned,2024-01-25,Frank Wright,False,"Laptop frequently overheating, causing concern..."
6,TCKT-1046,Mobile app not working properly,Software,App reinstalled and updated,2024-01-26,Gina Hall,True,Mobile application crashes every time it\'s op...
7,TCKT-1047,Shared drive access issue,Network,Drive access permissions corrected,2024-01-27,Harry Lopez,False,Issues with permissions preventing access to t...
8,TCKT-1048,Problem with emails marked as spam,Software,Email filter settings adjusted,2024-01-28,Ivy Taylor,True,"Important emails being sent to spam, missing c..."
9,TCKT-1049,PC blue screen error,Hardware,Operating system reinstalled,2024-01-29,Jack Martinez,False,"Repeated blue screen errors on PC, preventing ..."


In [7]:
from langchain_community.vectorstores import Chroma

In [10]:
import chromadb



chroma_client = chromadb.PersistentClient()
from langchain_community.embeddings.sentence_transformer import (
    SentenceTransformerEmbeddings,
)

# switch `create_collection` to `get_or_create_collection` to avoid creating a new collection every time
embedding_function = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")
df = old_tickets_df

def prepare_category_vector_db(category, df_category,collection_name):
    # Combine Issue and Description for embeddings
    df_category['combined_text'] = df_category['Issue'] + " " + df_category['Description']
    #df_category['Date'] = df_category['Date'].dt.strftime('%Y-%m-%d')
    # Generate embeddings
    #embeddings = embedding_model.encode(df_category['combined_text'].tolist())
    
    # Prepare metadata (e.g., Date)
    #metadata = df_category[['Date']].to_dict(orient='records')
    # Create a collection for the category
    
 
 
 
 
    Chroma.from_texts(
    texts=df_category['combined_text'].tolist(),
    embedding=embedding_function,
    collection_name=collection_name,
    client=chroma_client,
    ids=df_category['Ticket ID'].astype(str).tolist(),
    
)

# rocess each category in the dataset

categories = df.Category.unique()
for category in categories:
    df_category = df[df['Category'] == category]
    if category=='Network':
        collection_network = chroma_client.get_or_create_collection(
        name='Network'
    )
        collection_name='Network'

    elif category=='Software':
            collection_software = chroma_client.get_or_create_collection(
            name='Software'
        )
            collection_name='Software'
    elif category =='Hardware':
            collection_hardware = chroma_client.get_or_create_collection(
                name='Hardware'
            )
            collection_name='Hardware'
    elif category =='Account Management':
        collection_account_management = chroma_client.get_or_create_collection(
            name='Account_Management'
        )
        collection_name='Account_Management'
    prepare_category_vector_db(category, df_category,collection_name)

# Persist the database

print("Vector database created and persisted successfully.")

c:\Users\shiva\miniconda\envs\ticket\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\shiva\miniconda\envs\ticket\lib\site-packages\huggingface_hub\file_download.py:140: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Steve\.cache\huggingface\hub\models--sentence-transformers--all-MiniLM-L6-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In ord

Vector database created and persisted successfully.


In [11]:
results = collection_network.query(
    query_texts=["I have VPN connection issues"], # Chroma will embed this for you
    n_results=2 # how many results to return
)


C:\Users\Steve\.cache\chroma\onnx_models\all-MiniLM-L6-v2\onnx.tar.gz: 100%|██████████| 79.3M/79.3M [00:09<00:00, 8.93MiB/s]


In [12]:
results

{'ids': [['TCKT-1051', 'TCKT-1011']],
 'embeddings': None,
 'documents': [["VPN not connecting VPN fails to connect to the company\\'s network, showing error messages.",
   'VPN disconnection issues Repeated VPN disconnections impacting remote access to company network. This is a recurring issue.']],
 'uris': None,
 'data': None,
 'metadatas': [[None, None]],
 'distances': [[0.6393508069789297, 0.8015416584253541]],
 'included': [<IncludeEnum.distances: 'distances'>,
  <IncludeEnum.documents: 'documents'>,
  <IncludeEnum.metadatas: 'metadatas'>]}

In [13]:
new_tickets_df=pd.read_csv(r'data\new_tickets.csv')

In [14]:
new_tickets_df

,Ticket ID,Issue,Description,Category,Date
0,TCKT-2000,VPN connection timeout,VPN connection times out frequently during use.,Network,2024-02-09
1,TCKT-2001,Emails not syncing on mobile,User's emails are not syncing properly on thei...,Software,2024-02-10
2,TCKT-2002,New software installation request,A request to install new project management so...,Software,2024-02-11
3,TCKT-2003,Laptop screen flickering,The laptop screen starts flickering intermitte...,Hardware,2024-02-12
4,TCKT-2004,Cannot open files in shared drive,User reports an error when trying to open file...,Network,2024-02-13
5,TCKT-2005,Printer not connecting to WiFi,WiFi printer is not connecting to any devices ...,Hardware,2024-02-14
6,TCKT-2006,Application performance issue,Performance of the internal application is slo...,Software,2024-02-15
7,TCKT-2007,Lost password for multiple accounts,A user has lost passwords for multiple account...,Account Management,2024-02-16
8,TCKT-2008,Server access denied,User cannot access the server with their curre...,Network,2024-02-17
9,TCKT-2009,Spam filter blocking client emails,The company's spam filter is incorrectly block...,Software,2024-02-18


In [15]:
new_tickets_df['combined_text']=new_tickets_df['Issue'] + " " + new_tickets_df['Description']

In [16]:
[new_tickets_df['combined_text'].iloc[0]]

['VPN connection timeout VPN connection times out frequently during use.']

In [17]:
results=[]
for i in range(len(new_tickets_df)):
    if new_tickets_df['Category'].iloc[i]=='Network':
        results.append(collection_network.query(query_texts=[new_tickets_df['combined_text'].iloc[i]]))
    elif new_tickets_df['Category'].iloc[i]=='Software':
        results.append(collection_software.query(query_texts=[new_tickets_df['combined_text'].iloc[i]]))
    elif new_tickets_df['Category'].iloc[i]=='Hardware':
        results.append(collection_hardware.query(query_texts=[new_tickets_df['combined_text'].iloc[i]]))
    elif new_tickets_df['Category'].iloc[i]=='Account Management':
        results.append(collection_account_management.query(query_texts=[new_tickets_df['combined_text'].iloc[i]]))


Number of requested results 10 is greater than number of elements in index 8, updating n_results = 8
Number of requested results 10 is greater than number of elements in index 9, updating n_results = 9
Number of requested results 10 is greater than number of elements in index 8, updating n_results = 8
Number of requested results 10 is greater than number of elements in index 9, updating n_results = 9
Number of requested results 10 is greater than number of elements in index 2, updating n_results = 2
Number of requested results 10 is greater than number of elements in index 8, updating n_results = 8


In [18]:
results

[{'ids': [['TCKT-1011',
    'TCKT-1051',
    'TKT1006',
    'TKT1004',
    'TCKT-1047',
    'TKT1009',
    'TKT1007',
    'TCKT-1057']],
  'embeddings': None,
  'documents': [['VPN disconnection issues Repeated VPN disconnections impacting remote access to company network. This is a recurring issue.',
    "VPN not connecting VPN fails to connect to the company\\'s network, showing error messages.",
    'Wi-Fi keeps disconnecting The office Wi-Fi network keeps disconnecting, affecting multiple employees. Could be an issue with the Wi-Fi router or network configuration.',
    "Computer running slow User's computer is experiencing significant slowdowns, especially when running multiple applications. Might require performance diagnosis.",
    'Shared drive access issue Issues with permissions preventing access to the shared company drive. This is a recurring issue.',
    "Laptop battery not charging User's laptop battery is not charging when plugged in. Potential hardware issue with the ba